# Imports

In [1]:
# general purpose
import numpy as np
from itertools import combinations

# plotting
import matplotlib.pyplot as plt
import seaborn as sns

# calculations
from sklearn.metrics import pairwise_distances
from sklearn import metrics
from scipy.spatial.distance import cdist, pdist

# clustering
from sklearn.cluster import AgglomerativeClustering, KMeans, DBSCAN, AffinityPropagation
import hdbscan

# Loading data
Experiment 2 contains all 13 gestures in all 8 arm positions, 3 trials each

In [2]:
# select dataset and encoding type
dataName = 'allHV.npz'
emgHVType =  'hvRel'

allHV = np.load(dataName)

# extract data and labels based on gesture, trial, and position
hv = allHV[emgHVType]
gestLabel = allHV['gestLabel']
posLabel = allHV['posLabel']
trialLabel = allHV['trialLabel']

# get list of unique values for each label
gestures = np.unique(gestLabel)
positions = np.unique(posLabel)
trials = np.unique(trialLabel)

numGestures = len(gestures)
numPositions = len(positions)
numTrials = len(trials)

# get data size info
D = hv.shape[1] # hypervector dimension
numHV = 80 # number of examples per trial

# color palettes for plotting
gPalette = sns.color_palette('Paired', numGestures)
pPalette = sns.color_palette('Paired', numPositions)

# Load projection data for visualization
Used UMAP for reduction down to 2 dimensions

In [3]:
# performed across entire dataset
proj = np.load('./projections/projAll.npy')

# performed separately for each gesture 
projG = []
for g in gestures:
    projG.append(np.load('./projections/projG' + str(g) + '.npy'))
    
# performed separately for each arm position
projP = []
for p in positions:
    projP.append(np.load('./projections/projP' + str(p) + '.npy'))
    
# performed separately for each gesture/arm position pair
projGP = []
for g in gestures:
    temp = []
    for p in positions:
        temp.append(np.load('./projections/projG' + str(g) + 'P' + str(p) + '.npy'))
    projGP.append(temp)

# performed separately for each trial    
projGPT = []
for g in gestures:
    temp1 = []
    for p in positions:
        temp2 = []
        for t in trials:
            temp2.append(np.load('./projections/projG' + str(g) + 'P' + str(p) + 'T' + str(t) + '.npy'))
        temp1.append(temp2)
    projGPT.append(temp1)

# Helper functions

## General purpose HD functions

### bipolarize(X)
- Bipolarizes a set of hypervectors, where positive values are assigned +1, negative values are assigned -1, and zeros are randomly assigned either +/- 1
- Arguments:
    - X: hypervector numpy array with shape (numHV, D)
- Returns:
    - X: bipolarized hypervectors

In [1]:
def bipolarize(Y):
    X = np.copy(Y)
    X[X > 0] = 1.0
    X[X < 0] = -1.0
    X[X == 0] = np.random.choice([-1.0, 1.0], size=len(X[X == 0]))
    return X

### centroids(X, label)
- Gets the centroids of hypervectors based on cluster labels. If no label is specified, then return centroid of entire set of hypervectors
- Arguments:
    - X: hypervector numpy array with shape (numHV, D)
    - label: array with numHV labels
- Returns:
    - c: numpy array of centroids with shape (numCluster, D)

In [2]:
def centroids(X,label=None):
    if label is not None:
        c = np.zeros((len(np.unique(label)), X.shape[1]))
        for i,l in enumerate(np.unique(label)):
            c[i,:] = bipolarize(np.sum(X[label==l],axis=0))
    else:
        c = bipolarize(np.sum(X,axis=0)).reshape(1,-1)
    return c

## Labeling functions

### remove_singleton_clusters(clusterLabels):
- Takes all singleton clusters (containing only one object) and labels them as outliers (-1), renumbering resulting labels so they are contiguous
- Arguments:
    - clusterLabels: labels possibly containing singleton clusters
- Returns: 
    - clusterLabels: labels with all singleton clusters counted as outliers


In [6]:
def remove_singleton_clusters(clusterLabels):
    X = np.copy(clusterLabels)
    clusts = np.unique(X)
    for c in clusts:
        if len(X[X == c]) == 1:
            X[X == c] = -1.0
    clusts = np.unique(X[X != -1])
    for i,c in enumerate(clusts):
        X[X == c] = i
    return X

### make_outliers_singleton(clusterLabels):
- Takes all outliers, and gives them a valid singleton cluster label
- Arguments:
    - clusterLabels: labels possibly containing outliers
- Returns:
    - clusterLabels: labels with all outliers relabeled as valid singleton clusters

In [7]:
def make_outliers_singleton(clusterLabels):
    X = np.copy(clusterLabels)
    numOutliers = len(X[X == -1])
    X[X == -1] = np.arange(numOutliers) + max(X) + 1
    return X

# Clustering evaluation

### cluster_metrics(X, label, classLabel)
- Calculates both unsupervised and supervised clustering metrics on a particular clustering
- Arguments:
    - X: points being clustered
    - label: clustering labels
    - classLabel: supervised labels for comparison
- Returns:
    - out: dictionary containing all metrics, described below
    
### numClusts
Number of clusters including singleton clusters and outliers

### numOutliers
Number of outliers/singleton clusters

## Unsupervised metrics

### I1 (minimize)
Measure of overall graph-based cohesion. For each cluster, get average pairwise distance between points. Compute weighted sum over all clusters weighted by cluster size $m_i$.
\begin{equation*}
I_1 = \sum\limits_{i}{\frac{1}{m_i}\sum\limits_{x \in C_i, y \in C_i}{dist(x,y)}}
\end{equation*}

### I2 (minimize)
Measure of overall prototype-based cohesion. Sum the distances between each point and the centroid of the cluster it belongs to. 
\begin{equation*}
I_2 = \sum\limits_{i}{\sum\limits_{x \in C_i}{dist(x,c_i)}}
\end{equation*}

### E1 (maximize)
Measure of prototype-based separation. Sum the distances between each cluster prototype and the overall data centroid, weighted by cluster size. 
\begin{equation*}
E_1 = \sum\limits_{i}{m_i dist(c_i,c_{all})}
\end{equation*}

### G1 (maximize)
Metric combining graph-based separation and cohesion. Sum the distances between points in a cluster and all other clusters, and divide by the within-cluster cohesion.
\begin{equation*}
G_1 = \sum\limits_{i}{\frac{\sum\limits_{j,j\neq i}{\sum\limits_{x \in C_i, y \in C_j}{dist(x,y)}}}{\sum\limits_{x \in C_i, y \in C_i}{dist(x,y)}}}
\end{equation*}

### silScore (maximize)
Silhouette score, averaged across all objects. Effectively a normalized difference between distance to points within cluster and distance to points within closest separate cluster. Each object's silhouette score is computed as follows:
1. Compute the average distance between it and all objects in the same cluster. Call this $a_i$.
1. Compute the average distance between it and all objects in each other cluster. Get the minimum value and call this $b_i$.
The silhouette score for object $x_i$ is then:
\begin{equation*}
s_i = \frac{b_i - a_i}{max(a_i, b_i)}
\end{equation*}

### CHindex (maximize)
Calinski-Harabasz Index, or Variance Ratio Criterion. Ratio of the sum of between-clusters dispersion and of inter-cluster dispersion, where dispersion is the sum of distances squared.
\begin{equation*}
VRC = \frac{SS_B}{SS_W} \times \frac{N-k}{k-1}
\end{equation*}
where $SS_B$ is the between-cluster variance:
\begin{equation*}
SS_B = \sum\limits_{i\in k}{n_i ||m_i - m||^2}
\end{equation*}
$SS_W$ is the within-cluster variance:
\begin{equation*}
SS_W = \sum\limits_{i\in k}{\sum\limits_{x \in C_i}{||x - m_i||^2}}
\end{equation*}
$N$ is overall number of objects, $k$ is number of classes, $n_i$ is the number of objects in each class, $m_i$ is centroid of each class, and $m$ is centroid of all data. This is very similar to G1, except based on squared Euclidean distance instead of Hamming distance.

### DBscore (minimize)
Davies-Bouldin Index. Signifies the average similarity between clusters, where similarity compares distance between clusters with the size of clusters themselves.
\begin{equation*}
DB = \frac{1}{k}\sum\limits_{i \in k}{R_i} \\
R_i = \max\limits_{j \in k, j \neq i}{R_{ij}} \\
R_{ij} = \frac{s_i + s_j}{d_{ij}} \\
d_{ij} = dist(c_i, c_j) \\
s_i = \frac{1}{||c_i||}\sum\limits_{x \in C_i}{dist(x,c_i)}
\end{equation*}

## Supervised metrics

### entropy (minimize)
The degree to which each cluster consists of object of a single class. Based on clustering precision$p_{ij} = m_{ij}/m_j$ where number of objects $m$ in cluster $i$ and class $j$. Entropy for each cluster is calculated as $e_i = -\sum\limits_{j}{p_{ij}log_2p_{ij}}$

### purity (maximize)
Similar to entropy, where purity for each cluster is $p_i = \max\limits_{j}{p_{ij}}$

### fMeas (maximize)
F-measure, a combination of precision and recall, $r_{ij} = m_{ij}/m_j$. 
\begin{equation*}
F(i,j) = \frac{2\times p_{ij} \times r_{ij}}{p_{ij} + r_{ij}}
\end{equation*}
For overall score, take max for each cluster over classes, and sum weighted by cluster size.

### adjRand (maximize)
Rand index calculated using:
- $a$: the number of pairs of elements that are in the same set in clustering and in the same set in the supervised labels
- $b$: the number of pairs of elements that are in different sets in clustering and in different sets in the supervised labels

The raw, unadjsted form is:$RI = \frac{a+b}{C}$ where $C$ is the notal number of possible pairs. The adjusted version accounts for the score of random labelings $$ARI = \frac{RI - E[RI]}{\max(RI) - E[RI]}$$

### adjMI (maximize)
The level of mutual information shared between labeling and clustering. 

### homogeneity (maximize)
Metric for degree to which each cluster contains only memebrs of a single class.

### completeness (maximize)
Metric for degree to which all members of a given class are assigned to the same cluster.

### vMeas (maximize)
Harmonic mean of homogeneity and completeness.

### FMscore (maximize)
Geometric mean of pairwise precision and recall: $$FMI = \frac{TP}{\sqrt{(TP + FP)(TP + FN)}}$$ where $TP$ is the true positive rate (number of pairs of points that belong to the same clusters in both sets of labels), $FP$ is the false positive rate (number of pairs of points that belong to the same clusters in true labels but not predicted labels), $FN$ is false negative (number of pairs of points that belongs in the same clusters in the predicted labels but not in true labels).

In [8]:
def cluster_metrics(X,label,classLabel):
    
    out = {} # create output dictionary
    
    # relabel all outliers so that they become singleton clusters
    clustLabel = make_outliers_singleton(label)
    
    ## get unsupervised metrics
    
    clusts = np.unique(clustLabel) # first get a list of all unique cluster labels
    numClusts = len(clusts) # calculate the total number clusters
    outliers = [c for c in clusts if len(clustLabel[clustLabel==c]) == 1] # get all singleton clusters
    
    out['totalClusts'] = numClusts
    out['numOutliers'] = len(outliers)
    out['numClusts'] = numClusts - len(outliers)
    
    # first go through and calculate all of the within-cluster metrics
    m = np.zeros(numClusts) # number of objects in each cluster
    graphCo = np.zeros(numClusts) # graph-based cohesion
    protoCo = np.zeros(numClusts) # prototype-based cohesion
    protoSepSingle = np.zeros(numClusts) # prototype-based separation with centroid of entire set
    allCentroid = bipolarize(np.sum(X,axis=0)).reshape(1,-1) # centroid of entire set
    
    for c in clusts:
        m[c] = len(clustLabel[clustLabel==c])
#         graphCo[c] = max(np.sum(pairwise_distances(X[clustLabel==c],metric='hamming')), 1/X.shape[0])
        graphCo[c] = np.sum(pairwise_distances(X[clustLabel==c],metric='hamming'))
        if graphCo[c] == 0:
            graphCo[c] = X.shape[0]
        proto = bipolarize(np.sum(X[clustLabel==c],axis=0)).reshape(1,-1)
        protoCo[c] = np.sum(cdist(X[clustLabel==c],proto,'hamming'))
        protoSepSingle[c] = np.sum(cdist(allCentroid,proto,'hamming'))
    
    # calculate the overall metrics
    I1 = sum(graphCo/m)
    I2 = sum(protoCo)
    out['I1'] = I1
    out['I2'] = I2
    
    # now get separation metrics between pairs of clusters
    graphSep = np.zeros(len(list(combinations(clusts, 2)))) # graph-based separation
    protoSepPair = np.zeros(len(list(combinations(clusts, 2)))) # prototype-based separation between pairs of centroids
    graphSepSums = np.zeros(numClusts) # precalc for G1
    
    for c,comb in enumerate(combinations(clusts, 2)):
        X0 = X[clustLabel==comb[0]]
        X1 = X[clustLabel==comb[1]]
        graphSep[c] = np.sum(cdist(X0,X1,'hamming'))
        
        proto0 = bipolarize(np.sum(X0,axis=0)).reshape(1,-1)
        proto1 = bipolarize(np.sum(X1,axis=0)).reshape(1,-1)
        protoSepPair[c] = np.sum(cdist(proto0,proto1,'hamming'))
        
        graphSepSums[comb[0]] += graphSep[c]
        graphSepSums[comb[1]] += graphSep[c]
    
    # calcualte overall metrics
    E1 = sum(protoSepSingle*m)
    G1 = sum(graphSepSums/graphCo)
    
    out['E1'] = E1
    out['G1'] = G1
    
    # next compute holistic scores
    out['silScore'] = metrics.silhouette_score(X,clustLabel,'hamming')
    out['CHindex'] = metrics.calinski_harabasz_score(X, clustLabel)
    out['DBscore'] = metrics.davies_bouldin_score(X, clustLabel)
    
    # get supervised metrics
    
    classes = np.unique(classLabel) # next get a list of all unique class labels
    numClasses = len(classes) # number of actual classes
    
    e_i = np.zeros(numClusts)
    m_i = np.zeros(numClusts)
    p_i = np.zeros(numClusts)
    f_ij = np.zeros((numClusts,numClasses))
    m_j = np.zeros(numClasses)
    for i in clusts:
        m_i[i] = len(clustLabel[clustLabel==i])
        for j in classes:
            m_ij = len(clustLabel[(clustLabel==i) & (classLabel==j)])
            m_j[j] = len(clustLabel[classLabel==j])
            r_ij = m_ij/m_j[j]
            p_ij = m_ij/m_i[i]
            if m_ij > 0:
                f_ij[i,j] = 2*p_ij*r_ij/(p_ij + r_ij)
                e_i[i] += -p_ij*np.log2(p_ij)
                p_i[i] = max(p_ij,p_i[i])
                
    entropy = sum(e_i*m_i)/len(clustLabel)
    purity = sum(p_i*m_i)/len(clustLabel)
    fMeas = sum(np.max(f_ij,axis=0)*m_j)/len(clustLabel)
    
    out['entropy'] = entropy
    out['purity'] = purity
    out['fMeas'] = fMeas
    
    out['adjRand'] = metrics.adjusted_rand_score(clustLabel,classLabel)
    out['adjMI'] = metrics.adjusted_mutual_info_score(clustLabel,classLabel)
    out['homogeneity'] = metrics.homogeneity_score(clustLabel,classLabel)
    out['completeness'] = metrics.completeness_score(clustLabel,classLabel)
    out['vMeas'] = metrics.v_measure_score(clustLabel,classLabel)
    out['FMscore'] = metrics.fowlkes_mallows_score(clustLabel,classLabel)
    
    return out

In [9]:
def plot_cluster_metrics_1d(res,x,xName,clustName):
    
    met = {}
    met['totalClusts'] = ('Total number of clusters',min)
    met['numClusts'] = ('Number of non-singleton clusters',min)
    met['numOutliers'] = ('Number of outliers/singleton clusters',min)
    met['I1'] = ('Graph-based cohesion',min)
    met['I2'] = ('Prototype-based cohesion',min)
    met['E1'] = ('Prototype-based separation',max)
    met['G1'] = ('Graph-based separation/cohesion',max)
    met['silScore'] = ('Silhouette score',max)
    met['CHindex'] = ('Calinsky-Harabasz score',max)
    met['DBscore'] = ('Davies-Bouldin score',min)
    met['entropy'] = ('Entropy',min)
    met['purity'] = ('Purity',max)
    met['fMeas'] = ('F-Measure',max)
    met['adjRand'] = ('Adjusted Rand index',max)
    met['adjMI'] = ('Adjusted mutual information',max)
    met['homogeneity'] = ('Homogeneity',max)
    met['completeness'] = ('Completeness',max)
    met['vMeas'] = ('V-Measure',max)
    met['FMscore'] = ('Fowlkes-Mallows score',max)
    
    f,axes = plt.subplots(int(np.ceil(len(met.keys())/4)),4,figsize=(30,int(np.ceil(len(met.keys())/4))*6 + 4))
    f.subplots_adjust(hspace=0.3,wspace=0.3)

    f.suptitle('Cluster metrics - ' + clustName)

    axes = axes.flatten()
    for i,k in enumerate(met.keys()):
        axes[i].plot(x,res[k])
        axes[i].set(title=met[k][0],xlabel=xName,ylabel=k)
        
        extVal = met[k][1](res[k])
        extX = np.array(x)[(np.array(res[k]) == extVal)]
        extY = extVal*np.ones(len(extX))
        axes[i].scatter(extX,extY,marker='*',c='m',s=400)

    for ax in axes[len(met.keys())::]:
        ax.axis('off')

# Clustering algorithms

In [10]:
def clust_agglom_ward(X,n_clusters):
    cl = AgglomerativeClustering(n_clusters=n_clusters,
                                 affinity='euclidean',
                                 memory=None,
                                 connectivity=None,
                                 compute_full_tree='auto',
                                 linkage='ward',
                                 distance_threshold=None).fit(X)
    return cl.labels_

In [11]:
def clust_agglom_hamm(X,n_clusters):
    cl = AgglomerativeClustering(n_clusters=n_clusters,
                                 affinity='hamming',
                                 memory=None,
                                 connectivity=None,
                                 compute_full_tree='auto',
                                 linkage='average',
                                 distance_threshold=None).fit(X)
    return cl.labels_

In [12]:
def clust_kmeans_noinit(X,n_clusters):
    cl = KMeans(n_clusters=n_clusters,
                init='k-means++',
                n_init=100,
                max_iter=500,
                tol = 1e-4,
                precompute_distances='auto',
                verbose=0,
                random_state=None,
                copy_x=True,
                n_jobs=-1,
                algorithm='auto').fit(X)
    return cl.labels_

In [13]:
def clust_kmeans_init(X,n_clusters,init_clusters):
    cl = KMeans(n_clusters=n_clusters,
                init=init_clusters,
                n_init=1,
                max_iter=500,
                tol = 1e-4,
                precompute_distances='auto',
                verbose=0,
                random_state=None,
                copy_x=True,
                n_jobs=-1,
                algorithm='auto').fit(X)
    return cl.labels_

In [14]:
def clust_hdbscan(X,minClust,minSamp):
    cl = hdbscan.HDBSCAN(min_cluster_size=minClust,
                         min_samples=minSamp,
                         metric='hamming',
                         p=None,
                         alpha=None,
                         cluster_selection_epsilon=0.0,
                         algorithm='best',
                         leaf_size=40,
                         approx_min_span_tree=True,
                         gen_min_span_tree=False,
                         core_dist_n_jobs=-1,
                         cluster_selection_method='eom',
                         allow_single_cluster=False,
                         prediction_data=False,
                         match_reference_implementation=False).fit(X)
    softClusters = hdbscan.all_points_membership_vectors(cl)
    return np.array([np.argmax(x) for x in softClusters])

In [15]:
def clust_dbscan(X,eps,minSamp):
    cl = DBSCAN(eps=eps,
                min_samples=minSamp,
                metric='hamming',
                metric_params=None,
                algorithm='auto',
                leaf_size=30,
                p=None,
                n_jobs=-1).fit(X)
    return cl.labels_

In [16]:
def clust_hdc(Y,t,perm=None):
    label = -np.ones(Y.shape[0],dtype='int32')
    clusts = []
    
    if perm is not None:
        X = np.copy(Y[perm])
    else:
        X = np.copy(Y)
    
    for i,x in enumerate(X):
        if not clusts:
            clusts.append(0)
            label[i] = 0
        else:
            sim = np.zeros(len(clusts))
            for c in clusts:
                proto = bipolarize(np.sum(X[label==c],axis=0)).reshape(1,-1)
                sim[c] = 1 - cdist(proto,x.reshape(1,-1),'hamming')
            if np.max(sim) > t:
                label[i] = np.argmax(sim)
            else:
                label[i] = max(clusts) + 1
                clusts.append(max(clusts) + 1)
                
    if perm is not None:
        return label[np.argsort(perm)]
    else:
        return label

In [57]:
def clust_affinity(X,prefScale,damping):
    affinity = -pairwise_distances(X,metric='hamming')
    pref = prefScale*(np.min(affinity) - np.median(affinity)) + np.median(affinity)
    cl = AffinityPropagation(damping=damping,
                             max_iter=500,
                             convergence_iter=15,
                             copy=True,
                             preference=pref,
                             affinity='precomputed',
                             verbose=0).fit(affinity)
    return cl.labels_

# Generating Labels

In [18]:
def test_num_clusts(X,func,superLabels,n_clusters):
    res = []
    lab = []
    for n in n_clusters:
        labels = func(X,n)
        lab.append(labels)
        res.append(cluster_metrics(X,labels,superLabels))
        
    out = {}
    for k in res[0].keys():
        out[k] = [d[k] for d in res]
        
    return out, lab

In [19]:
def test_kmeans_init_num_clusts(X,func,superLabels,n_clusters):
    res = []
    lab = []
    for n in n_clusters:
        startLabels = make_outliers_singleton(func(X,n))
        init_clusters = centroids(X,startLabels)
        labels = clust_kmeans_init(X,n,init_clusters)
        lab.append(labels)
        res.append(cluster_metrics(X,labels,superLabels))
        
    out = {}
    for k in res[0].keys():
        out[k] = [d[k] for d in res]
        
    return out, lab

In [20]:
def test_hdc_thresh(X,superLabels,thresh):
    res = []
    lab = []
    validThresh = []
    for t in thresh:
        labels = clust_hdc(X,t)
        if (len(np.unique(labels)) > 1) and (len(np.unique(labels)) < X.shape[0]):
            validThresh.append(t)
            lab.append(labels)
            res.append(cluster_metrics(X,labels,superLabels))
    
    out = {}
    for k in res[0].keys():
        out[k] = [d[k] for d in res]
        
    return out, lab, np.array(validThresh)

In [120]:
def labels_hdc(X,verbose=False, random=False):
    
    numIter = 100
    labels = {}
    
    tUpper = 1
    
    tMax = 1
    tMin = 0.5
    t = tMin
    n = 0
    
    lowestClust = X.shape[0] + 1
    if random:
        for i in range(numIter):
            if verbose:
                print("\rRuning random permutation %d out of %d" % (i+1, numIter), end =" ") 
            temp = clust_hdc(X,t,np.random.permutation(X.shape[0]))
            if len(np.unique(temp)) < lowestClust:
                lab = temp
    else:
        lab = clust_hdc(X,t)
    
    labels[t] = lab

    clusts = np.unique(lab)
    numClust = len(clusts)
    outliers = [c for c in clusts if len(lab[lab==c]) == 1]
    numNonSingle = numClust - len(outliers)
    
    if verbose:
        if random:
            print(', threshold = %f: %d total clusters, %d non-singleton' % (t, numClust, numNonSingle))
        else:
            print('Iteration %d, threshold = %f: %d total clusters, %d non-singleton' % (n, t, numClust, numNonSingle))

    while (numNonSingle != 2) or (numClust > round(X.shape[0]/2)):
        n += 1
        if numNonSingle == 1:
            tMin = t
            t = np.mean([t,tMax])
        else:
            tMax = t
            t = np.mean([t,tMin])
            
        lowestClust = X.shape[0] + 1
        if random:
            if verbose:
                print("\rRuning random permutation %d out of %d" % (i+1, numIter), end =" ") 
            for i in range(numIter):
                temp = clust_hdc(X,t,np.random.permutation(X.shape[0]))
                if len(np.unique(temp)) < lowestClust:
                    lab = temp
        else:
            lab = clust_hdc(X,t)
        
        labels[t] = lab

        clusts = np.unique(lab)
        numClust = len(clusts)
        outliers = [c for c in clusts if len(lab[lab==c]) == 1]
        numNonSingle = numClust - len(outliers)
        
        if verbose:
            if random:
                print(', threshold = %f: %d total clusters, %d non-singleton' % (t, numClust, numNonSingle))
            else:
                print('Iteration %d, threshold = %f: %d total clusters, %d non-singleton' % (n, t, numClust, numNonSingle))
            
        if numClust > X.shape[0]/2:
            tUpper = min(tUpper, t)
            
    N = 20
    for t in np.linspace(t + (tUpper-t)/(N+1),tUpper,N,endpoint=False):
        lowestClust = X.shape[0] + 1
        if random:
            for i in range(numIter):
                if verbose:
                    print("\rRuning random permutation %d out of %d" % (i+1, numIter), end =" ") 
                temp = clust_hdc(X,t,np.random.permutation(X.shape[0]))
                if len(np.unique(temp)) < lowestClust:
                    lab = temp
        else:
            lab = clust_hdc(X,t)
        
        labels[t] = lab

        clusts = np.unique(lab)
        numClust = len(clusts)
        outliers = [c for c in clusts if len(lab[lab==c]) == 1]
        numNonSingle = numClust - len(outliers)
        
        if verbose:
            if random:
                print(', threshold = %f: %d total clusters, %d non-singleton' % (t, numClust, numNonSingle))
            else:
                print('Iteration %d, threshold = %f: %d total clusters, %d non-singleton' % (n, t, numClust, numNonSingle))
    
    return labels
        
   

In [118]:
def labels_affinity_prop(X,verbose=False):
    
    labels = {}
    
    for d in [0.5, 0.7, 0.9, 0.95, 0.97, 0.98, 0.99]:
        currPref = 0
        n = 0
        lab = clust_affinity(X,currPref,d)
        labels[(d,currPref)] = lab

        clusts = np.unique(lab)
        numClust = len(clusts)
        outliers = [c for c in clusts if len(lab[lab==c]) == 1]
        numNonSingle = numClust - len(outliers)
        
        if verbose:
            print('Iteration %d, preference scale = %d, damping = %f: %d total clusters, %d non-singleton' % (n, currPref, d, numClust, numNonSingle))

        currPref = 1
        n = 1
        lab = clust_affinity(X,currPref,d)
        labels[(d,currPref)] = lab

        clusts = np.unique(lab)
        numClust = len(clusts)
        outliers = [c for c in clusts if len(lab[lab==c]) == 1]
        numNonSingle = numClust - len(outliers)
        
        if verbose:
            print('Iteration %d, preference scale = %d, damping = %f: %d total clusters, %d non-singleton' % (n, currPref, d, numClust, numNonSingle))

        while numNonSingle > 1:
            n += 1
            currPref = currPref*2
            lab = clust_affinity(X,currPref,d)
            labels[(d,currPref)] = lab

            clusts = np.unique(lab)
            numClust = len(clusts)
            outliers = [c for c in clusts if len(lab[lab==c]) == 1]
            numNonSingle = numClust - len(outliers)
            
            if verbose:
                print('Iteration %d, preference scale = %d, damping = %f: %d total clusters, %d non-singleton' % (n, currPref, d, numClust, numNonSingle))

        step = currPref/4
        while step >= 0.1:
            if numNonSingle == 1:
                currPref -= step
            else:
                currPref += step
            n += 1

            lab = clust_affinity(X,currPref,d)
            labels[(d,currPref)] = lab  

            clusts = np.unique(lab)
            numClust = len(clusts)
            outliers = [c for c in clusts if len(lab[lab==c]) == 1]
            numNonSingle = numClust - len(outliers)
            
            if verbose:
                print('Iteration %d, preference scale = %f, damping = %f: %d total clusters, %d non-singleton' % (n, currPref, d, numClust, numNonSingle))

            step /= 2
            
    return labels
    

# Runs

In [121]:
g = 0
# p = 0
# filt = (gestLabel == g) & (posLabel == p)
filt = (gestLabel == g)
X = hv[filt]

# l = labels_affinity_prop(X,verbose=False)
l = labels_hdc(X,verbose=True,random=True)

Runing random permutation 100 out of 100 , threshold = 0.500000: 1 total clusters, 1 non-singleton
Runing random permutation 100 out of 100 , threshold = 0.750000: 31 total clusters, 16 non-singleton
Runing random permutation 100 out of 100 , threshold = 0.625000: 7 total clusters, 7 non-singleton
Runing random permutation 100 out of 100 , threshold = 0.562500: 2 total clusters, 2 non-singleton
Runing random permutation 100 out of 100 , threshold = 0.583333: 4 total clusters, 4 non-singleton
Runing random permutation 100 out of 100 , threshold = 0.604167: 5 total clusters, 5 non-singleton
Runing random permutation 100 out of 100 , threshold = 0.625000: 6 total clusters, 6 non-singleton
Runing random permutation 100 out of 100 , threshold = 0.645833: 8 total clusters, 8 non-singleton
Runing random permutation 100 out of 100 , threshold = 0.666667: 8 total clusters, 8 non-singleton
Runing random permutation 100 out of 100 , threshold = 0.687500: 10 total clusters, 10 non-singleton
Runing

KeyboardInterrupt: 

In [24]:
# trialSubset = trialLabel[filt]
# gestSubset = gestLabel[filt]
# posSubset = posLabel[filt]

# combs, superLabels = np.unique(np.column_stack((trialSubset,gestSubset,posSubset)),axis=0,return_inverse=True)